In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import PyImbalReg as pir
from sklearn.model_selection import train_test_split

In [2]:
df_boston_tr = pd.read_csv(r".\CSV_data\boston_train.csv")
############### val

df_boston_val = pd.read_csv(r".\CSV_data\boston_val.csv")

############### test

df_boston_test = pd.read_csv(r".\CSV_data\boston_test.csv")

### boston dtaset

In [3]:
#%%  train-val-test
x_boston_train, y_boston_train = df_boston_tr.iloc[:,:-1], df_boston_tr.iloc[:,-1]
x_boston_val, y_boston_val = df_boston_val.iloc[:,:-1], df_boston_val.iloc[:,-1]
x_boston_test, y_boston_test = df_boston_test.iloc[:,:-1], df_boston_test.iloc[:,-1]

In [4]:
#%%  RF
from sklearn.ensemble import RandomForestRegressor
rf_boston = RandomForestRegressor(n_estimators=50,  random_state=1)
rf_boston.fit(x_boston_train, y_boston_train)
rf_boston_pred = rf_boston.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(rf_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 11.992	L1 2.622	G-Mean 1.672
 * Many: MSE 4.544	L1 1.749	G-Mean 1.256
 * Median: MSE 14.784	L1 3.024	G-Mean 1.957
 * Low: MSE 22.620	L1 3.852	G-Mean 2.681
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [5]:
#%%  xgb 
import xgboost as xgb
xgb_boston = xgb.XGBRegressor(n_estimators=50,
                            random_state=1)
xgb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
xgb_boston_pred = xgb_boston.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(xgb_boston_pred, y_boston_test, y_boston_train)

[0]	validation_0-rmse:20.09190
[1]	validation_0-rmse:15.15440
[2]	validation_0-rmse:12.02874
[3]	validation_0-rmse:9.79174
[4]	validation_0-rmse:8.29615
[5]	validation_0-rmse:7.10603
[6]	validation_0-rmse:6.31079
[7]	validation_0-rmse:5.78598
[8]	validation_0-rmse:5.42601
[9]	validation_0-rmse:5.17437
[10]	validation_0-rmse:4.93989
[11]	validation_0-rmse:4.80405
[12]	validation_0-rmse:4.70943
[13]	validation_0-rmse:4.61439
[14]	validation_0-rmse:4.54872
[15]	validation_0-rmse:4.51594
[16]	validation_0-rmse:4.50559
[17]	validation_0-rmse:4.46168
[18]	validation_0-rmse:4.45564
[19]	validation_0-rmse:4.43043
[20]	validation_0-rmse:4.42563
[21]	validation_0-rmse:4.42830
[22]	validation_0-rmse:4.44235
[23]	validation_0-rmse:4.42957
[24]	validation_0-rmse:4.41562
[25]	validation_0-rmse:4.41511
[26]	validation_0-rmse:4.41173
[27]	validation_0-rmse:4.39062
[28]	validation_0-rmse:4.39589
[29]	validation_0-rmse:4.38673
[30]	validation_0-rmse:4.38569
[31]	validation_0-rmse:4.38389
[32]	validation

#### smogn

In [9]:
# smogn sample
import smogn
import resreg
relevance_pdf = resreg.pdf_relevance(y_boston_train, bandwidth=0.5)
np.random.seed(1)
boston_smogn = smogn.smoter(data = pd.concat([x_boston_train,y_boston_train],axis = 1),
                        y = 'HousValue',                               #  need change  columns!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                        rel_thres = 0.8,
                       )
boston_smogn.to_csv(r".\Ca_Data\smogn_boston_train.csv")

r_index: 100%|########################################################################| 18/18 [00:00<00:00, 311.18it/s]


In [10]:
#%%  smogn train-val-test
x_boston_train_smogn, y_boston_train_smogn = boston_smogn.iloc[:,:-1], boston_smogn.iloc[:,-1]
x_boston_train_smogn.head(),x_boston_train_smogn.shape

(       CRIM         ZN     INDUS  CHAS       NOX        RM        AGE  \
 0  0.118764  48.808248  3.289955     0  0.434410  6.587913  28.048924   
 1  0.108813  37.728402  5.191794     0  0.434356  6.837609  21.563980   
 2  0.752482  16.079780  4.407105     0  0.619558  6.936380  83.855158   
 3  0.585565   0.000000  6.200000     0  0.505997  7.322776  81.535884   
 4  0.716409  22.043799  4.219343     0  0.626562  7.039854  80.962038   
 
         DIS  RAD  TAX    PTRATIO           B      LSTAT  
 0  4.802045    4  398  15.337097  383.910727   4.560000  
 1  5.178475    6  352  15.794949  387.857403   4.758316  
 2  2.356257    5  272  13.862448  386.582821  13.380681  
 3  3.145562    8  307  17.400000  391.914725   6.606556  
 4  2.404212    4  262  13.470074  384.597548  13.896860  ,
 (418, 13))

In [76]:
#%%  RF + smogn
from sklearn.ensemble import RandomForestRegressor
rf_boston_smogn = RandomForestRegressor(n_estimators=50, max_features=0.8, random_state=1)
rf_boston_smogn.fit(x_boston_train_smogn, y_boston_train_smogn)
rf_boston_smogn_pred = rf_boston_smogn.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(rf_boston_smogn_pred, y_boston_test, y_boston_train)

 * Overall: MSE 44.820	L1 4.837	G-Mean 2.549
 * Many: MSE 15.981	L1 2.800	G-Mean 2.051
 * Median: MSE 53.391	L1 5.407	G-Mean 3.002
 * Low: MSE 88.227	L1 8.109	G-Mean 5.999
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [77]:
#%%  xgb + smogn
import xgboost as xgb
xgb_boston_smogn = xgb.XGBRegressor(n_estimators=50,
                                    colsample_bytree = 0.8,
                            random_state=1)
xgb_boston_smogn.fit(x_boston_train_smogn, y_boston_train_smogn,  eval_set=[(x_boston_val, y_boston_val)])
xgb_boston_smogn_pred = xgb_boston_smogn.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(xgb_boston_smogn_pred, y_boston_test, y_boston_train)

[0]	validation_0-rmse:18.84478
[1]	validation_0-rmse:13.46056
[2]	validation_0-rmse:9.17380
[3]	validation_0-rmse:7.28249
[4]	validation_0-rmse:5.69288
[5]	validation_0-rmse:4.86963
[6]	validation_0-rmse:4.54907
[7]	validation_0-rmse:4.52378
[8]	validation_0-rmse:4.60107
[9]	validation_0-rmse:4.62189
[10]	validation_0-rmse:4.76607
[11]	validation_0-rmse:4.78625
[12]	validation_0-rmse:4.83351
[13]	validation_0-rmse:4.81398
[14]	validation_0-rmse:4.88041
[15]	validation_0-rmse:4.97473
[16]	validation_0-rmse:5.05882
[17]	validation_0-rmse:5.10317
[18]	validation_0-rmse:5.07078
[19]	validation_0-rmse:5.10785
[20]	validation_0-rmse:5.10594
[21]	validation_0-rmse:5.13564
[22]	validation_0-rmse:5.13563
[23]	validation_0-rmse:5.13103
[24]	validation_0-rmse:5.13054
[25]	validation_0-rmse:5.13403
[26]	validation_0-rmse:5.13213
[27]	validation_0-rmse:5.15409
[28]	validation_0-rmse:5.16189
[29]	validation_0-rmse:5.16256
[30]	validation_0-rmse:5.17263
[31]	validation_0-rmse:5.17392
[32]	validation_

### smoter

In [17]:
import resreg
relevance_pdf = resreg.pdf_relevance(y_boston_train, bandwidth=0.5)
x_boston_train_smoter, y_boston_train_smoter = resreg.smoter(x_boston_train,y_boston_train, relevance = relevance_pdf, 
                                relevance_threshold=0.8, k=7, over='balance', random_state=1)
boston_smoter = pd.concat([pd.DataFrame(data = x_boston_train_smoter,columns = x_boston_train.columns), 
                       pd.DataFrame(data = y_boston_train_smoter,columns = ['HousValue'])],axis = 1)    #  need change columns!!!!!!!!!!
boston_smoter.to_csv(r".\Ca_Data\smoter_boston_train.csv")

In [18]:
#%%  smoter train-val-test
x_boston_train_smoter, y_boston_train_smoter = boston_smoter.iloc[:,:-1], boston_smoter.iloc[:,-1]
x_boston_train_smoter.head(),x_boston_train_smoter.shape

(     CRIM   ZN  INDUS  CHAS    NOX     RM    AGE    DIS   RAD    TAX  PTRATIO  \
 0   2.780  0.0  19.58   0.0  0.871  4.903   97.8  1.346   5.0  403.0     14.7   
 1  17.867  0.0  18.10   0.0  0.671  6.223  100.0  1.386  24.0  666.0     20.2   
 2  88.976  0.0  18.10   0.0  0.671  6.968   91.9  1.416  24.0  666.0     20.2   
 3  15.874  0.0  18.10   0.0  0.671  6.545   99.1  1.519  24.0  666.0     20.2   
 4  16.812  0.0  18.10   0.0  0.700  5.277   98.1  1.426  24.0  666.0     20.2   
 
          B  LSTAT  
 0  396.898  29.29  
 1  393.738  21.78  
 2  396.898  17.21  
 3  396.898  21.08  
 4  396.898  30.81  ,
 (351, 13))

In [78]:
#%%  RF + smoter
from sklearn.ensemble import RandomForestRegressor
rf_boston_smoter = RandomForestRegressor(n_estimators=50, max_features=0.8, random_state=1)
rf_boston_smoter.fit(x_boston_train_smoter, y_boston_train_smoter)
rf_boston_smoter_pred = rf_boston_smoter.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(rf_boston_smoter_pred, y_boston_test, y_boston_train)

 * Overall: MSE 14.332	L1 2.925	G-Mean 2.070
 * Many: MSE 4.965	L1 1.951	G-Mean 1.496
 * Median: MSE 15.119	L1 2.891	G-Mean 1.843
 * Low: MSE 32.448	L1 4.995	G-Mean 4.106
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [134]:
#%%  xgb + smoter
import xgboost as xgb
xgb_boston_smoter = xgb.XGBRegressor(n_estimators=50,
                                     #colsample_bytree = 0.8,
                            random_state=1)
xgb_boston_smoter.fit(x_boston_train_smoter, y_boston_train_smoter,  eval_set=[(x_boston_val, y_boston_val)])
xgb_boston_smoter_pred = xgb_boston_smoter.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(xgb_boston_smoter_pred, y_boston_test, y_boston_train)

[0]	validation_0-rmse:19.74369
[1]	validation_0-rmse:14.74566
[2]	validation_0-rmse:11.19818
[3]	validation_0-rmse:8.64108
[4]	validation_0-rmse:6.99429
[5]	validation_0-rmse:5.75756
[6]	validation_0-rmse:4.92407
[7]	validation_0-rmse:4.28953
[8]	validation_0-rmse:3.99008
[9]	validation_0-rmse:3.71470
[10]	validation_0-rmse:3.59010
[11]	validation_0-rmse:3.52582
[12]	validation_0-rmse:3.43220
[13]	validation_0-rmse:3.39988
[14]	validation_0-rmse:3.37217
[15]	validation_0-rmse:3.37248
[16]	validation_0-rmse:3.37879
[17]	validation_0-rmse:3.37541
[18]	validation_0-rmse:3.36881
[19]	validation_0-rmse:3.36942
[20]	validation_0-rmse:3.35952
[21]	validation_0-rmse:3.36917
[22]	validation_0-rmse:3.35207
[23]	validation_0-rmse:3.34638
[24]	validation_0-rmse:3.34628
[25]	validation_0-rmse:3.33762
[26]	validation_0-rmse:3.33403
[27]	validation_0-rmse:3.33970
[28]	validation_0-rmse:3.33303
[29]	validation_0-rmse:3.33745
[30]	validation_0-rmse:3.33512
[31]	validation_0-rmse:3.33897
[32]	validation

#### WERCS

In [21]:
import resreg
relevance_pdf = resreg.pdf_relevance(y_boston_train, bandwidth=0.5)
x_boston_train_wercs, y_boston_train_wercs = resreg.wercs(x_boston_train,y_boston_train, relevance = relevance_pdf, 
                                under=0.5, over=0.5,delta = 0.02, random_state=1)
boston_wercs = pd.concat([pd.DataFrame(data = x_boston_train_wercs,columns = x_boston_train.columns), 
                       pd.DataFrame(data = y_boston_train_wercs,columns = ['HousValue'])],axis = 1)                #  change columns!!!!
boston_wercs.to_csv(r".\Ca_Data\wercs_boston_train.csv")

In [22]:
#%%  wercs train-val-test
x_boston_train_wercs, y_boston_train_wercs = boston_wercs.iloc[:,:-1], boston_wercs.iloc[:,-1]
x_boston_train_wercs.head(),x_boston_train_wercs.shape

(     CRIM    ZN  INDUS  CHAS    NOX     RM   AGE    DIS   RAD    TAX  PTRATIO  \
 0   0.020  95.0   2.68   0.0  0.416  8.034  31.9  5.118   4.0  224.0     14.7   
 1  88.976   0.0  18.10   0.0  0.671  6.968  91.9  1.416  24.0  666.0     20.2   
 2   0.006  18.0   2.31   0.0  0.538  6.575  65.2  4.090   1.0  296.0     15.3   
 3   1.127   0.0  19.58   1.0  0.871  5.012  88.0  1.610   5.0  403.0     14.7   
 4   0.128  12.5   6.07   0.0  0.409  5.885  33.0  6.498   4.0  345.0     18.9   
 
          B  LSTAT  
 0  390.549   2.88  
 1  396.898  17.21  
 2  396.898   4.98  
 3  343.279  12.12  
 4  396.898   8.79  ,
 (352, 13))

In [80]:
#%%  RF + wercs
from sklearn.ensemble import RandomForestRegressor
rf_boston_wercs = RandomForestRegressor(n_estimators=50, max_features=0.8, random_state=1)
rf_boston_wercs.fit(x_boston_train_wercs, y_boston_train_wercs)
rf_boston_wercs_pred = rf_boston_wercs.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(rf_boston_wercs_pred, y_boston_test, y_boston_train)

 * Overall: MSE 13.078	L1 2.807	G-Mean 1.800
 * Many: MSE 4.869	L1 1.893	G-Mean 1.573
 * Median: MSE 13.430	L1 2.844	G-Mean 1.849
 * Low: MSE 27.129	L1 4.584	G-Mean 3.985
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [81]:
#%%  xgb + wercs
import xgboost as xgb
xgb_boston_wercs = xgb.XGBRegressor(n_estimators=50,
                                    colsample_bytree = 0.8,
                            random_state=1)
xgb_boston_wercs.fit(x_boston_train_wercs, y_boston_train_wercs,  eval_set=[(x_boston_val, y_boston_val)])
xgb_boston_wercs_pred = xgb_boston_wercs.predict(x_boston_test)
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(xgb_boston_wercs_pred, y_boston_test, y_boston_train)

[0]	validation_0-rmse:20.52453
[1]	validation_0-rmse:15.94117
[2]	validation_0-rmse:12.00574
[3]	validation_0-rmse:9.32553
[4]	validation_0-rmse:7.55116
[5]	validation_0-rmse:6.26905
[6]	validation_0-rmse:5.58821
[7]	validation_0-rmse:5.05407
[8]	validation_0-rmse:4.60878
[9]	validation_0-rmse:4.28340
[10]	validation_0-rmse:4.10705
[11]	validation_0-rmse:3.99046
[12]	validation_0-rmse:3.88968
[13]	validation_0-rmse:3.84728
[14]	validation_0-rmse:3.81380
[15]	validation_0-rmse:3.78729
[16]	validation_0-rmse:3.74799
[17]	validation_0-rmse:3.70965
[18]	validation_0-rmse:3.68604
[19]	validation_0-rmse:3.65628
[20]	validation_0-rmse:3.64585
[21]	validation_0-rmse:3.62911
[22]	validation_0-rmse:3.62184
[23]	validation_0-rmse:3.61365
[24]	validation_0-rmse:3.61458
[25]	validation_0-rmse:3.61407
[26]	validation_0-rmse:3.62264
[27]	validation_0-rmse:3.60928
[28]	validation_0-rmse:3.60295
[29]	validation_0-rmse:3.60424
[30]	validation_0-rmse:3.59936
[31]	validation_0-rmse:3.59933
[32]	validation

### rebagg-smoter

In [71]:
from sklearn.tree import DecisionTreeRegressor
import resreg
import warnings
warnings.filterwarnings('ignore')
relevance_pdf = resreg.pdf_relevance(y_boston_train, bandwidth=0.5)
reg_boston_tree = DecisionTreeRegressor(max_features=0.8, 
                                        random_state=1)  # base learner tree

rebagg_boston_tree = resreg.Rebagg(m=50, s=0.8, base_reg=reg_boston_tree) # create a rebagg instance

rebagg_boston_tree.fit(x_boston_train, y_boston_train, relevance=relevance_pdf, relevance_threshold=0.8, 
                    sample_method='smoter', size_method="balance", random_state=1)

rebagg_boston_tree_pred = rebagg_boston_tree.predict(x_boston_test)

In [72]:
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(rebagg_boston_tree_pred, y_boston_test, y_boston_train)

 * Overall: MSE 11.083	L1 2.573	G-Mean 1.729
 * Many: MSE 5.168	L1 1.847	G-Mean 1.400
 * Median: MSE 15.004	L1 3.006	G-Mean 1.888
 * Low: MSE 13.442	L1 3.045	G-Mean 2.159
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


## Imr-GB

In [37]:
import numpy as np
import torch
from torch.distributions import MultivariateNormal as MVN
import torch.nn.functional as F
from sklearn.mixture import GaussianMixture
y_train = y_boston_train                  #  prior distribution!!!!
# gradient
def get_gmm(dist, n_components):
    # fit a **ground truth** label distribution
    all_labels = torch.tensor(dist)
    if len(all_labels.shape) == 1:
        all_labels = all_labels.unsqueeze(-1)
    gmm = GaussianMixture(n_components=n_components).fit(all_labels)
    gmm_dict = {'means': gmm.means_, 'weights': gmm.weights_, 'variances': gmm.covariances_}
    return gmm_dict
def gradient_gai(y_pred, y_true):
    gmm = get_gmm(dist= y_train, n_components=60 # GaussianMixture components
                 )
    gmm = {k: torch.tensor(gmm[k]) for k in gmm}
    noise_var = 1.
    target, pred = torch.tensor(y_true).reshape(-1,1), torch.tensor(y_pred).reshape(-1,1).requires_grad_()
    I = torch.eye(target.shape[-1])
    mse_term = -MVN(target, noise_var*I).log_prob(pred)
    balancing_term = MVN(gmm['means'], gmm['variances']+noise_var*I).log_prob(pred.unsqueeze(1)) + gmm['weights'].log()
    balancing_term = torch.logsumexp(balancing_term, dim=1)
    loss = mse_term + balancing_term
    loss = (loss * (2 * noise_var)).sum()
    loss.backward()
    grad_pred = pred.grad
    grad_pred = np.array(grad_pred).reshape(-1,1)
    return grad_pred
"""def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1"""
def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1
def Im_bmse_loss(y_true,y_pred):
    grad = gradient_gai(y_pred, y_true)
    hess = hessian_gai(y_pred, y_true)
    return grad, hess

In [38]:
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,
                             #max_depth = 5,
                             #colsample_bytree = 0.8,
                             objective = Im_bmse_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)

[0]	validation_0-rmse:15.31917
[1]	validation_0-rmse:9.26960
[2]	validation_0-rmse:6.91872
[3]	validation_0-rmse:6.42622
[4]	validation_0-rmse:6.19818
[5]	validation_0-rmse:6.09918
[6]	validation_0-rmse:5.84508
[7]	validation_0-rmse:5.76429
[8]	validation_0-rmse:5.76009
[9]	validation_0-rmse:5.73523
[10]	validation_0-rmse:5.70671
[11]	validation_0-rmse:5.70171
[12]	validation_0-rmse:5.69667
[13]	validation_0-rmse:5.69646
[14]	validation_0-rmse:5.68695
[15]	validation_0-rmse:5.66450
[16]	validation_0-rmse:5.65631
[17]	validation_0-rmse:5.65086
[18]	validation_0-rmse:5.65716
[19]	validation_0-rmse:5.65782
[20]	validation_0-rmse:5.65793
[21]	validation_0-rmse:5.66415
[22]	validation_0-rmse:5.66558
[23]	validation_0-rmse:5.67161
[24]	validation_0-rmse:5.67970
[25]	validation_0-rmse:5.67779
[26]	validation_0-rmse:5.67947
[27]	validation_0-rmse:5.67890
[28]	validation_0-rmse:5.67910
[29]	validation_0-rmse:5.68014
[30]	validation_0-rmse:5.67955
[31]	validation_0-rmse:5.67748
[32]	validation_0

In [39]:
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 20.850	L1 3.379	G-Mean 2.119
 * Many: MSE 7.000	L1 2.080	G-Mean 1.476
 * Median: MSE 28.336	L1 4.036	G-Mean 2.556
 * Low: MSE 30.107	L1 4.624	G-Mean 3.177
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [40]:
import numpy as np
import torch
from torch.distributions import MultivariateNormal as MVN
import torch.nn.functional as F
from sklearn.mixture import GaussianMixture
y_train = y_boston_train               #  prior distribution
# gradient
def get_gmm(dist, n_components):
    # fit a **ground truth** label distribution
    all_labels = torch.tensor(dist)
    if len(all_labels.shape) == 1:
        all_labels = all_labels.unsqueeze(-1)
    gmm = GaussianMixture(n_components=n_components).fit(all_labels)
    gmm_dict = {'means': gmm.means_, 'weights': gmm.weights_, 'variances': gmm.covariances_}
    return gmm_dict
def gradient_gai(y_pred, y_true):
    gmm = get_gmm(dist= y_train, n_components=10         # GaussianMixture components
                 )
    gmm = {k: torch.tensor(gmm[k]) for k in gmm}
    noise_var = 1.
    target, pred = torch.tensor(y_true).reshape(-1,1), torch.tensor(y_pred).reshape(-1,1).requires_grad_()
    I = torch.eye(target.shape[-1])
    mse_term = -MVN(target, noise_var*I).log_prob(pred)
    balancing_term = MVN(gmm['means'], gmm['variances']+noise_var*I).log_prob(pred.unsqueeze(1)) + gmm['weights'].log()
    balancing_term = torch.logsumexp(balancing_term, dim=1)
    loss = mse_term + balancing_term
    loss = (loss * (2 * noise_var)).sum()
    loss.backward()
    grad_pred = pred.grad
    grad_pred = np.array(grad_pred).reshape(-1,1)
    return grad_pred
"""def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1"""
def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1
def Im_bmse_loss(y_true,y_pred):
    grad = gradient_gai(y_pred, y_true)
    hess = hessian_gai(y_pred, y_true)
    return grad, hess

In [41]:
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,
                             #max_depth = 5,
                             #colsample_bytree = 0.8,
                             objective = Im_bmse_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)

[0]	validation_0-rmse:14.34312
[1]	validation_0-rmse:8.72741
[2]	validation_0-rmse:5.94290
[3]	validation_0-rmse:5.17660
[4]	validation_0-rmse:4.93416
[5]	validation_0-rmse:4.73398
[6]	validation_0-rmse:4.63269
[7]	validation_0-rmse:4.64788
[8]	validation_0-rmse:4.62369
[9]	validation_0-rmse:4.64502
[10]	validation_0-rmse:4.65226
[11]	validation_0-rmse:4.62975
[12]	validation_0-rmse:4.62903
[13]	validation_0-rmse:4.61961
[14]	validation_0-rmse:4.59143
[15]	validation_0-rmse:4.57419
[16]	validation_0-rmse:4.56936
[17]	validation_0-rmse:4.56411
[18]	validation_0-rmse:4.57364
[19]	validation_0-rmse:4.57659
[20]	validation_0-rmse:4.58692
[21]	validation_0-rmse:4.59996
[22]	validation_0-rmse:4.60023
[23]	validation_0-rmse:4.60740
[24]	validation_0-rmse:4.60394
[25]	validation_0-rmse:4.60858
[26]	validation_0-rmse:4.60839
[27]	validation_0-rmse:4.60651
[28]	validation_0-rmse:4.60819
[29]	validation_0-rmse:4.60623
[30]	validation_0-rmse:4.61062
[31]	validation_0-rmse:4.61081
[32]	validation_0

In [42]:
#%%   shot metrics
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 21.809	L1 3.472	G-Mean 2.360
 * Many: MSE 7.482	L1 2.138	G-Mean 1.345
 * Median: MSE 28.186	L1 3.927	G-Mean 2.723
 * Low: MSE 35.804	L1 5.188	G-Mean 3.975
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [43]:
import numpy as np
import torch
from torch.distributions import MultivariateNormal as MVN
import torch.nn.functional as F
from sklearn.mixture import GaussianMixture
y_train = y_boston_train               #  prior distribution
# gradient
def get_gmm(dist, n_components):
    # fit a **ground truth** label distribution
    all_labels = torch.tensor(dist)
    if len(all_labels.shape) == 1:
        all_labels = all_labels.unsqueeze(-1)
    gmm = GaussianMixture(n_components=n_components).fit(all_labels)
    gmm_dict = {'means': gmm.means_, 'weights': gmm.weights_, 'variances': gmm.covariances_}
    return gmm_dict
def gradient_gai(y_pred, y_true):
    gmm = get_gmm(dist= y_train, n_components=1    # GaussianMixture components
                 )
    gmm = {k: torch.tensor(gmm[k]) for k in gmm}
    noise_var = 1.
    target, pred = torch.tensor(y_true).reshape(-1,1), torch.tensor(y_pred).reshape(-1,1).requires_grad_()
    I = torch.eye(target.shape[-1])
    mse_term = -MVN(target, noise_var*I).log_prob(pred)
    balancing_term = MVN(gmm['means'], gmm['variances']+noise_var*I).log_prob(pred.unsqueeze(1)) + gmm['weights'].log()
    balancing_term = torch.logsumexp(balancing_term, dim=1)
    loss = mse_term + balancing_term
    loss = (loss * (2 * noise_var)).sum()
    loss.backward()
    grad_pred = pred.grad
    grad_pred = np.array(grad_pred).reshape(-1,1)
    return grad_pred
"""def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1"""
def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1
def Im_bmse_loss(y_true,y_pred):
    grad = gradient_gai(y_pred, y_true)
    hess = hessian_gai(y_pred, y_true)
    return grad, hess

In [44]:
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,
                             #max_depth = 5,
                             #colsample_bytree = 0.8,
                             objective = Im_bmse_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)

[0]	validation_0-rmse:13.54811
[1]	validation_0-rmse:7.33568
[2]	validation_0-rmse:5.33176
[3]	validation_0-rmse:4.79329
[4]	validation_0-rmse:4.34442
[5]	validation_0-rmse:4.36126
[6]	validation_0-rmse:4.31200
[7]	validation_0-rmse:4.19478
[8]	validation_0-rmse:4.16267
[9]	validation_0-rmse:4.13913
[10]	validation_0-rmse:4.16150
[11]	validation_0-rmse:4.14856
[12]	validation_0-rmse:4.12006
[13]	validation_0-rmse:4.10836
[14]	validation_0-rmse:4.10341
[15]	validation_0-rmse:4.07223
[16]	validation_0-rmse:4.06799
[17]	validation_0-rmse:4.06128
[18]	validation_0-rmse:4.06131
[19]	validation_0-rmse:4.05569
[20]	validation_0-rmse:4.05448
[21]	validation_0-rmse:4.04160
[22]	validation_0-rmse:4.04324
[23]	validation_0-rmse:4.04170
[24]	validation_0-rmse:4.04076
[25]	validation_0-rmse:4.04337
[26]	validation_0-rmse:4.04140
[27]	validation_0-rmse:4.03862
[28]	validation_0-rmse:4.03649
[29]	validation_0-rmse:4.03569
[30]	validation_0-rmse:4.03522
[31]	validation_0-rmse:4.03306
[32]	validation_0

In [45]:
#%%   shot metrics  1
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 24.020	L1 3.430	G-Mean 1.945
 * Many: MSE 7.162	L1 2.039	G-Mean 1.327
 * Median: MSE 34.169	L1 4.240	G-Mean 2.894
 * Low: MSE 36.614	L1 4.709	G-Mean 3.034
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [46]:
import numpy as np
import torch
from torch.distributions import MultivariateNormal as MVN
import torch.nn.functional as F
from sklearn.mixture import GaussianMixture
y_train = y_boston_train                  #  prior distribution
# gradient
def get_gmm(dist, n_components):
    # fit a **ground truth** label distribution
    all_labels = torch.tensor(dist)
    if len(all_labels.shape) == 1:
        all_labels = all_labels.unsqueeze(-1)
    gmm = GaussianMixture(n_components=n_components).fit(all_labels)
    gmm_dict = {'means': gmm.means_, 'weights': gmm.weights_, 'variances': gmm.covariances_}
    return gmm_dict
def gradient_gai(y_pred, y_true):
    gmm = get_gmm(dist= y_train, n_components=3 # GaussianMixture components
                 )
    gmm = {k: torch.tensor(gmm[k]) for k in gmm}
    noise_var = 1.
    target, pred = torch.tensor(y_true).reshape(-1,1), torch.tensor(y_pred).reshape(-1,1).requires_grad_()
    I = torch.eye(target.shape[-1])
    mse_term = -MVN(target, noise_var*I).log_prob(pred)
    balancing_term = MVN(gmm['means'], gmm['variances']+noise_var*I).log_prob(pred.unsqueeze(1)) + gmm['weights'].log()
    balancing_term = torch.logsumexp(balancing_term, dim=1)
    loss = mse_term + balancing_term
    loss = (loss * (2 * noise_var)).sum()
    loss.backward()
    grad_pred = pred.grad
    grad_pred = np.array(grad_pred).reshape(-1,1)
    return grad_pred
"""def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1"""
def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1
def Im_bmse_loss(y_true,y_pred):
    grad = gradient_gai(y_pred, y_true)
    hess = hessian_gai(y_pred, y_true)
    return grad, hess

In [47]:
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,
                             #max_depth = 5,
                             #colsample_bytree = 0.8,
                             objective = Im_bmse_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)

[0]	validation_0-rmse:13.75126
[1]	validation_0-rmse:7.46211
[2]	validation_0-rmse:5.47361
[3]	validation_0-rmse:4.65419
[4]	validation_0-rmse:4.43486
[5]	validation_0-rmse:4.21981
[6]	validation_0-rmse:4.18832
[7]	validation_0-rmse:4.22433
[8]	validation_0-rmse:4.21851
[9]	validation_0-rmse:4.20951
[10]	validation_0-rmse:4.19807
[11]	validation_0-rmse:4.20260
[12]	validation_0-rmse:4.20165
[13]	validation_0-rmse:4.19522
[14]	validation_0-rmse:4.17225
[15]	validation_0-rmse:4.13525
[16]	validation_0-rmse:4.13549
[17]	validation_0-rmse:4.12234
[18]	validation_0-rmse:4.13020
[19]	validation_0-rmse:4.13730
[20]	validation_0-rmse:4.11132
[21]	validation_0-rmse:4.11178
[22]	validation_0-rmse:4.10084
[23]	validation_0-rmse:4.06662
[24]	validation_0-rmse:4.06652
[25]	validation_0-rmse:4.06587
[26]	validation_0-rmse:4.07309
[27]	validation_0-rmse:4.07229
[28]	validation_0-rmse:4.07155
[29]	validation_0-rmse:4.06858
[30]	validation_0-rmse:4.06903
[31]	validation_0-rmse:4.02857
[32]	validation_0

In [48]:
#%%   shot metrics  3
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 25.039	L1 3.521	G-Mean 2.055
 * Many: MSE 7.010	L1 1.983	G-Mean 1.392
 * Median: MSE 35.347	L1 4.406	G-Mean 2.661
 * Low: MSE 39.448	L1 4.843	G-Mean 2.437
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [78]:
import numpy as np
import torch
from torch.distributions import MultivariateNormal as MVN
import torch.nn.functional as F
from sklearn.mixture import GaussianMixture
y_train = y_boston_train               #  prior distribution
# gradient
def get_gmm(dist, n_components):
    # fit a **ground truth** label distribution
    all_labels = torch.tensor(dist)
    if len(all_labels.shape) == 1:
        all_labels = all_labels.unsqueeze(-1)
    gmm = GaussianMixture(n_components=n_components).fit(all_labels)
    gmm_dict = {'means': gmm.means_, 'weights': gmm.weights_, 'variances': gmm.covariances_}
    return gmm_dict
def gradient_gai(y_pred, y_true):
    gmm = get_gmm(dist= y_train, n_components=5 # GaussianMixture components
                 )
    gmm = {k: torch.tensor(gmm[k]) for k in gmm}
    noise_var = 1.
    target, pred = torch.tensor(y_true).reshape(-1,1), torch.tensor(y_pred).reshape(-1,1).requires_grad_()
    I = torch.eye(target.shape[-1])
    mse_term = -MVN(target, noise_var*I).log_prob(pred)
    balancing_term = MVN(gmm['means'], gmm['variances']+noise_var*I).log_prob(pred.unsqueeze(1)) + gmm['weights'].log()
    balancing_term = torch.logsumexp(balancing_term, dim=1)
    loss = mse_term + balancing_term
    loss = (loss * (2 * noise_var)).sum()
    loss.backward()
    grad_pred = pred.grad
    grad_pred = np.array(grad_pred).reshape(-1,1)
    return grad_pred
"""def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1"""
def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1
def Im_bmse_loss(y_true,y_pred):
    grad = gradient_gai(y_pred, y_true)
    hess = hessian_gai(y_pred, y_true)
    return grad, hess

In [79]:
#%%%   Imr-gb   5
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,
                             #max_depth = 5,
                             #colsample_bytree = 0.8,
                             objective = Im_bmse_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)

[0]	validation_0-rmse:14.02316
[1]	validation_0-rmse:8.54772
[2]	validation_0-rmse:6.06241
[3]	validation_0-rmse:5.22621
[4]	validation_0-rmse:5.09168
[5]	validation_0-rmse:5.00406
[6]	validation_0-rmse:4.97401
[7]	validation_0-rmse:4.89291
[8]	validation_0-rmse:4.88905
[9]	validation_0-rmse:4.89366
[10]	validation_0-rmse:4.89702
[11]	validation_0-rmse:4.84442
[12]	validation_0-rmse:4.84229
[13]	validation_0-rmse:4.83087
[14]	validation_0-rmse:4.82425
[15]	validation_0-rmse:4.81031
[16]	validation_0-rmse:4.80595
[17]	validation_0-rmse:4.80786
[18]	validation_0-rmse:4.80556
[19]	validation_0-rmse:4.79128
[20]	validation_0-rmse:4.78954
[21]	validation_0-rmse:4.79127
[22]	validation_0-rmse:4.79271
[23]	validation_0-rmse:4.77899
[24]	validation_0-rmse:4.78109
[25]	validation_0-rmse:4.78273
[26]	validation_0-rmse:4.77984
[27]	validation_0-rmse:4.77731
[28]	validation_0-rmse:4.77589
[29]	validation_0-rmse:4.77682
[30]	validation_0-rmse:4.77726
[31]	validation_0-rmse:4.77393
[32]	validation_0

In [80]:
#%%   shot metrics  5
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 21.370	L1 3.401	G-Mean 2.210
 * Many: MSE 7.643	L1 2.250	G-Mean 1.726
 * Median: MSE 28.738	L1 3.983	G-Mean 2.582
 * Low: MSE 31.427	L1 4.511	G-Mean 3.130
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [73]:
import numpy as np
import torch
from torch.distributions import MultivariateNormal as MVN
import torch.nn.functional as F
from sklearn.mixture import GaussianMixture
y_train = y_boston_train               #  prior distribution
# gradient
def get_gmm(dist, n_components):
    # fit a **ground truth** label distribution
    all_labels = torch.tensor(dist)
    if len(all_labels.shape) == 1:
        all_labels = all_labels.unsqueeze(-1)
    gmm = GaussianMixture(n_components=n_components).fit(all_labels)
    gmm_dict = {'means': gmm.means_, 'weights': gmm.weights_, 'variances': gmm.covariances_}
    return gmm_dict
def gradient_gai(y_pred, y_true):
    gmm = get_gmm(dist= y_train, n_components=7 # GaussianMixture components
                 )
    gmm = {k: torch.tensor(gmm[k]) for k in gmm}
    noise_var = 1.
    target, pred = torch.tensor(y_true).reshape(-1,1), torch.tensor(y_pred).reshape(-1,1).requires_grad_()
    I = torch.eye(target.shape[-1])
    mse_term = -MVN(target, noise_var*I).log_prob(pred)
    balancing_term = MVN(gmm['means'], gmm['variances']+noise_var*I).log_prob(pred.unsqueeze(1)) + gmm['weights'].log()
    balancing_term = torch.logsumexp(balancing_term, dim=1)
    loss = mse_term + balancing_term
    loss = (loss * (2 * noise_var)).sum()
    loss.backward()
    grad_pred = pred.grad
    grad_pred = np.array(grad_pred).reshape(-1,1)
    return grad_pred
"""def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1"""
def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1
def Im_bmse_loss(y_true,y_pred):
    grad = gradient_gai(y_pred, y_true)
    hess = hessian_gai(y_pred, y_true)
    return grad, hess

In [76]:
#%%%   Imr-gb   
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,
                             #max_depth = 5,
                             #colsample_bytree = 0.8,
                             objective = Im_bmse_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)

[0]	validation_0-rmse:14.33445
[1]	validation_0-rmse:7.54352
[2]	validation_0-rmse:5.39643
[3]	validation_0-rmse:4.50554
[4]	validation_0-rmse:4.28987
[5]	validation_0-rmse:4.10452
[6]	validation_0-rmse:4.02792
[7]	validation_0-rmse:3.98276
[8]	validation_0-rmse:3.96033
[9]	validation_0-rmse:3.95652
[10]	validation_0-rmse:3.98080
[11]	validation_0-rmse:3.97950
[12]	validation_0-rmse:3.94580
[13]	validation_0-rmse:3.95871
[14]	validation_0-rmse:3.95635
[15]	validation_0-rmse:3.95173
[16]	validation_0-rmse:3.96005
[17]	validation_0-rmse:3.96877
[18]	validation_0-rmse:3.96874
[19]	validation_0-rmse:3.96519
[20]	validation_0-rmse:3.96037
[21]	validation_0-rmse:3.96018
[22]	validation_0-rmse:3.96722
[23]	validation_0-rmse:3.97106
[24]	validation_0-rmse:3.97263
[25]	validation_0-rmse:3.97775
[26]	validation_0-rmse:3.97636
[27]	validation_0-rmse:3.97119
[28]	validation_0-rmse:3.96877
[29]	validation_0-rmse:3.96724
[30]	validation_0-rmse:3.96482
[31]	validation_0-rmse:3.96644
[32]	validation_0

In [77]:
#%%   shot metrics   7
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 21.112	L1 3.305	G-Mean 1.838
 * Many: MSE 7.950	L1 2.143	G-Mean 1.488
 * Median: MSE 30.785	L1 3.987	G-Mean 2.140
 * Low: MSE 27.536	L1 4.229	G-Mean 2.552
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


### Imr-bay-gb

In [9]:
#%%%   Imr-gb
import numpy as np
import torch
from torch.distributions import MultivariateNormal as MVN
import torch.nn.functional as F
from sklearn.mixture import BayesianGaussianMixture
y_train = y_boston_train               #  prior distribution!!!!
n_components = int(0.1*len(y_train))   # up boundary
# gradient
def get_bgm(dist):
    # fit a **ground truth** label distribution
    all_labels = torch.tensor(dist)
    if len(all_labels.shape) == 1:
        all_labels = all_labels.unsqueeze(-1)
    bgm = BayesianGaussianMixture(n_components= n_components, weight_concentration_prior_type="dirichlet_process", 
                                  #mean_precision_prior=0.8,
                                  init_params="random",
                                  random_state=1).fit(all_labels)
    bgm_dict = {'means': bgm.means_, 'weights': bgm.weights_, 'variances': bgm.covariances_}
    return bgm_dict
def gradient_gai(y_pred, y_true):
    bgm = get_bgm(dist= y_train)
    bgm = {k: torch.tensor(bgm[k]) for k in bgm}
    noise_var = 1.
    target, pred = torch.tensor(y_true).reshape(-1,1), torch.tensor(y_pred).reshape(-1,1).requires_grad_()
    I = torch.eye(target.shape[-1])
    mse_term = -MVN(target, noise_var*I).log_prob(pred)
    balancing_term = MVN(bgm['means'], bgm['variances']+noise_var*I).log_prob(pred.unsqueeze(1)) + bgm['weights'].log()
    balancing_term = torch.logsumexp(balancing_term, dim=1)
    loss = mse_term + balancing_term
    loss = (loss * (2 * noise_var)).sum()
    loss.backward()
    grad_pred = pred.grad
    grad_pred = np.array(grad_pred).reshape(-1,1)
    return grad_pred
"""def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1"""
def hessian_gai(y_pred, y_true):
    y_true, y_pred = np.array(y_true).reshape(-1,1), np.array(y_pred).reshape(-1,1)
    return 0*(y_true + y_pred) + 1
def Im_bay_loss(y_true,y_pred):
    grad = gradient_gai(y_pred, y_true)
    hess = hessian_gai(y_pred, y_true)
    return grad, hess

In [10]:
#%%%   Imr-bay-gb
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,

                             objective = Im_bay_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)

[0]	validation_0-rmse:13.68604


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[1]	validation_0-rmse:7.41368


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[2]	validation_0-rmse:5.45350


...

D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[48]	validation_0-rmse:4.19024


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[49]	validation_0-rmse:4.19051


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


In [11]:
#%%   shot metrics  0.1
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

 * Overall: MSE 22.579	L1 3.544	G-Mean 2.417
 * Many: MSE 6.738	L1 2.034	G-Mean 1.430
 * Median: MSE 32.846	L1 4.498	G-Mean 3.228
 * Low: MSE 32.886	L1 4.656	G-Mean 3.522
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


In [7]:
#%%%   Imr-bay-gb
import xgboost as xgb
Imr_gb_boston = xgb.XGBRegressor(n_estimators=50,
                             objective = Im_bay_loss,
                             #eval_metric = mean_absolute_error
                             random_state=1
                            )
Imr_gb_boston.fit(x_boston_train, y_boston_train,  eval_set=[(x_boston_val, y_boston_val)])
Imr_gb_boston_pred = Imr_gb_boston.predict(x_boston_test)
#%%   shot metrics  0.1
import shot_metric
shot_metric.val_metrics(Imr_gb_boston_pred, y_boston_test, y_boston_train)

[0]	validation_0-rmse:13.66250


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[1]	validation_0-rmse:7.37970


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[2]	validation_0-rmse:5.34726


...

D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[48]	validation_0-rmse:4.23410


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


[49]	validation_0-rmse:4.23470
 * Overall: MSE 22.192	L1 3.391	G-Mean 1.973
 * Many: MSE 5.127	L1 1.834	G-Mean 1.252
 * Median: MSE 31.209	L1 4.046	G-Mean 2.253
 * Low: MSE 37.777	L1 5.072	G-Mean 3.638
 * Many_shot 27.000	Median 32.000	Low_shot 17.000


D:\Users\admin\anaconda3\lib\site-packages\sklearn\mixture\_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
